<a href="https://colab.research.google.com/github/ewylka/coursera-test/blob/master/TBM_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructions:
1. Change the date under Choose_date
2. Press play and scroll down to see the preview table.
3. Check table if ok.
4. Press play in code box nr.2 to create excel
5. Download excel 😀
5. Thank data analyst Ewelina

In [ ]:
Choose_date='2024-06-03'  #Choose date, format yyyy-mm-dd

#Automatic, nie dotykaj
from google.colab import drive
import pandas as pd
from datetime import datetime
from google.colab import data_table
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/00 TBM DATA/Oslo E6 TBM_New Template_202463-600 usa.csv' #File path, mozesz podmienic (z google drive)
df = pd.read_csv(file_path,  delimiter=';', on_bad_lines='skip') #Delimiter fix, dont touch
df.replace('--empty--', '', inplace=True) #replaces empty cells

#Filter cells you dont need
df = df[df['Critical'] != 'No']
df = df[~df['Code'].isin(['DO', 'RB'])]

#converts 'Start' to datetime format
df['Start'] = pd.to_datetime(df['Start'])

#function to adjust shift day from 6am-6am
def calculate_shift_day(start):
    if start.hour >= 6:  # If hour is 6:00 AM or later
        return start.date()
    else:  # If hour is before 6:00 AM
        return (start - pd.Timedelta(days=1)).date()

# Function result to create new Shift Day column
df['Shift Day'] = df['Start'].apply(calculate_shift_day)
specific_shift_day = datetime.strptime(Choose_date, '%Y-%m-%d').date()  # Specific shift day from header, must be a date object
DailyDate_df = df[df['Shift Day'] == specific_shift_day] #filter out rows with the specific shift day

#filter rows with the chosen shift day & only the desired columns
desired_columns = ['Description', 'Code', 'Critical', 'Duration in min', 'Shift Day']
DailyDate_df = df[df['Shift Day'] == specific_shift_day][desired_columns]

#sum of the 'Duration in min' per day
duration_sum_per_activity = DailyDate_df.groupby(['Description', 'Code','Critical']).agg({'Duration in min': 'sum'}).reset_index()

new_description_column = f'Description of activity on Shift Day {specific_shift_day}:' # Rename the 'Description' column
duration_sum_per_activity.rename(columns={'Description': new_description_column}, inplace=True)
print(f"Total Duration in min for each activity on Shift Day {specific_shift_day}. Check if correct before exporting excel in box under.")

#show interactive data table
data_table.DataTable(duration_sum_per_activity, include_index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total Duration in min for each activity on Shift Day 2024-06-03. Check if correct before exporting excel in box under.


,Description of activity on Shift Day 2024-06-03:,Code,Critical,Duration in min
0,,Adv,,60
1,,Regrip,,14
2,Bullflex installation (grouting),260,Yes,1103
3,DT - On TBM -Cleaning Telescopic Shield,650,Yes,78
4,Injection drilling,130,Yes,128
5,Other Maintenance / Delay,860,Yes,10
6,TBM Breakdown - Electrical System,510,Yes,40


Press play to download excel:

In [ ]:
#export the excel file

file_name = f'{specific_shift_day} daily TBM rapport.xlsx' #excel file name
duration_sum_per_activity.to_excel(file_name, index=False) #save to excel
print(f"""DataFrame exported to '{file_name}' successfully.

Download link:""")

def create_download_link(file_name):
    import base64
    from IPython.display import HTML
    with open(file_name, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode()

    return HTML(f'<a download="{file_name}" href="data:text/csv;base64,{b64}" target="_blank">Click here to download {file_name}</a>')

create_download_link(file_name)

DataFrame exported to '2024-06-03 daily TBM rapport.xlsx' successfully.

Download link:
